# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator
import random
from collections import defaultdict

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks
def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.591756174871633),
 (1, 2.591756174871633),
 (2, 2.591756174871633),
 (3, 2.591756174871633),
 (4, 2.591756174871633)]

## Inverted index 

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  #print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

[(0, [('', 6), ('banana', 2), ('is', 18)]),
 (1, [('a', 2), ('it', 18), ('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

[(0,
  [(None, 0.02193139569855418),
   (None, 0.04666629170541636),
   (None, 0.11287687508967459),
   (None, 0.1665943452836528),
   (None, 0.16840982485693812),
   (None, 0.24431816689092478),
   (None, 0.25992331279019065),
   (None, 0.28118458443304695),
   (None, 0.298761144060279),
   (None, 0.33138113787686185),
   (None, 0.42516446054930757),
   (None, 0.43090455723152477),
   (None, 0.44668586502950725),
   (None, 0.4641662804567319)]),
 (1,
  [(None, 0.5891362352119163),
   (None, 0.6036460817020096),
   (None, 0.6361066872443863),
   (None, 0.6375678555309057),
   (None, 0.6709740677179076),
   (None, 0.6738223450758172),
   (None, 0.6980500117978639),
   (None, 0.7129761558124712),
   (None, 0.7989839357824207),
   (None, 0.810515087554536),
   (None, 0.8411614077248419),
   (None, 0.8868828877399885),
   (None, 0.9230415122002741),
   (None, 0.9350681440436049),
   (None, 0.9802260416931836),
   (None, 0.9924533995207883)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
from typing import List, Iterator

# Переопределение функции RECORDREADER для чтения списка чисел
def RECORDREADER(input_list: List[int]):
    # Все числа будут иметь одинаковый ключ, так как нам нужно найти максимум по всему списку
    return [(1, number) for number in input_list]

# Переопределение функции MAP
def MAP(_, number:int):
    # Просто передаем число дальше
    yield (1, number)
    
# Переопределение функции REDUCE
def REDUCE(_, numbers:Iterator[int]):
    # Находим максимальное число
    max_number = max(numbers)
    yield (_, max_number)

# Определение функции flatten
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

# Определение функции groupbykey
def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

# Входной список чисел
input_list = [10, 15, 3, 7, 9, 25, 16]

# Исполнение MapReduce алгоритма
def MapReduce(input_list, RECORDREADER, MAP, REDUCE):
    # Подготавливаем данные через RECORDREADER
    prepared_data = RECORDREADER(input_list)
    # Применяем MAP к каждому элементу
    mapped = flatten(map(lambda x: MAP(*x), prepared_data))
    # Группируем по ключу и применяем REDUCE
    reduced = flatten(map(lambda x: REDUCE(*x), groupbykey(mapped)))
    return list(reduced)

# Запуск MapReduce алгоритма
result = MapReduce(input_list, RECORDREADER, MAP, REDUCE)
print(result[0][1])

25


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
def RECORDREADER_average(input_list: List[int]):
    # Все числа будут иметь одинаковый ключ, так как нам нужно вычислить общее среднее
    return [(1, number) for number in input_list]

# Переопределение функции MAP для вычисления среднего
def MAP_average(_, number:int):
    # Передаем число и счетчик 1 для каждого числа
    yield (1, (number, 1))
    
# Переопределение функции REDUCE для вычисления среднего
def REDUCE_average(_, values:Iterator[tuple]):
    sum_numbers, count_numbers = 0, 0
    for value, count in values:
        sum_numbers += value
        count_numbers += count
    # Возвращаем среднее значение
    if count_numbers > 0:
        yield (_, sum_numbers / count_numbers)
    else:
        yield (_, 0)

# Входной список чисел
input_list_average = [10, 20, 30, 40, 50]

# Повторное выполнение MapReduce алгоритма для вычисления среднего
result_average = MapReduce(input_list_average, RECORDREADER_average, MAP_average, REDUCE_average)
print(result_average[0][1])

30.0


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
from collections import defaultdict
import random

def map_func(pair):
    num1, num2 = pair
    return num1, num2

def reduce_func(key, nums):
    total = sum(nums)
    count = len(nums)
    return total / count

def data_generator(count):
    result = []
    for _ in range(count):
        num1 = 1
        num2 = random.randint(0, 100)
        result.append((num1, num2))
    return result

map_output = list(map(map_func, data_generator(100)))
print("Map output:", map_output, '\n')

shuffle_output = defaultdict(list)
for key, value in map_output:
    shuffle_output[key].append(value)
shuffle_output = list(shuffle_output.items())
print("Shuffle output:", shuffle_output, '\n')

reduce_output = [reduce_func(key, nums) for key, nums in shuffle_output]
print("Reduce output:", reduce_output, '\n')

Map output: [(1, 24), (1, 91), (1, 50), (1, 15), (1, 88), (1, 60), (1, 22), (1, 99), (1, 85), (1, 44), (1, 13), (1, 73), (1, 55), (1, 100), (1, 86), (1, 25), (1, 82), (1, 89), (1, 8), (1, 81), (1, 40), (1, 2), (1, 91), (1, 66), (1, 38), (1, 87), (1, 15), (1, 12), (1, 35), (1, 6), (1, 11), (1, 98), (1, 97), (1, 6), (1, 50), (1, 82), (1, 41), (1, 53), (1, 88), (1, 61), (1, 22), (1, 76), (1, 23), (1, 9), (1, 88), (1, 17), (1, 72), (1, 46), (1, 56), (1, 100), (1, 7), (1, 39), (1, 76), (1, 65), (1, 53), (1, 63), (1, 64), (1, 53), (1, 55), (1, 12), (1, 53), (1, 63), (1, 23), (1, 8), (1, 6), (1, 64), (1, 81), (1, 75), (1, 84), (1, 43), (1, 76), (1, 98), (1, 39), (1, 38), (1, 7), (1, 49), (1, 0), (1, 70), (1, 47), (1, 10), (1, 48), (1, 85), (1, 19), (1, 28), (1, 72), (1, 4), (1, 43), (1, 94), (1, 25), (1, 24), (1, 0), (1, 54), (1, 4), (1, 80), (1, 94), (1, 95), (1, 99), (1, 41), (1, 2), (1, 65)] 

Shuffle output: [(1, [24, 91, 50, 15, 88, 60, 22, 99, 85, 44, 13, 73, 55, 100, 86, 25, 82, 89, 8,

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        """Yields a key-value pair of document ID and line"""
        for doc_id, document in enumerate(split):
            for line_id, line in enumerate(document.strip().split('\n')):
                yield f"{doc_id}:{line_id}", line

    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        split = documents[i:i+split_size]
        yield RECORDREADER(split)  # Используем функцию напрямую

def MAP(doc_id: str, line: str) -> Iterator[tuple]:
    """Yields key-value pairs of words"""
    return ((word, word) for word in line.split())

def REDUCE(key: str, values: Iterator[str]) -> Iterator[str]:
    """Yields unique keys"""
    yield key
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(partitioned_output)

[(0, ['banana', 'is']), (1, ['a', 'it', 'what'])]


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
def MAP(elements):
    return [(t, [t]) for t in elements if t[0] > 50 and t[0] < 55] 

def REDUCE(mapped_items):
    return [value[0] for values in mapped_items for value in values]

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

record = RECORDREADER(10)
print(record, '\n')
parts = 5
record_partitional = [record[d:d + parts] for d in range(0, len(record), parts)]

print(REDUCE([MAP(x) for x in record_partitional]))

[(45, 70, 31), (63, 46, 80), (71, 12, 81), (33, 85, 71), (5, 100, 94), (90, 69, 88), (50, 52, 19), (32, 60, 6), (54, 40, 74), (69, 56, 25)] 

[(54, 40, 74)]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
S = {1, 12, 56}  

def MAP(t):
  res_list = [element for element in t if element in S]  
  res = tuple(res_list)
  return (res, res)

def REDUCE(key, values):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  return {k: [v for v in vs if v is not None] for k, vs in iterable}.items()  # Using dictionary comprehension

record = RECORDREADER(50)
print("Records: ",record, '\n')

map_out = [MAP(x) for x in record]  # Using list comprehension
print("Mapped Data: ",map_out, '\n')

shuffle_out = list(group_by_key(map_out))
print("Shuffled Data: ",shuffle_out, '\n')

reduce_out = [REDUCE(*x)[0] for x in shuffle_out]  # Using list comprehension
print("Reduced Data: ",reduce_out, '\n')

Records:  [(18, 23, 12), (17, 78, 7), (83, 41, 28), (84, 58, 10), (39, 82, 19), (60, 10, 24), (44, 100, 32), (32, 35, 97), (15, 31, 73), (90, 0, 73), (21, 71, 81), (74, 96, 13), (68, 89, 34), (98, 21, 61), (35, 14, 69), (36, 64, 2), (96, 35, 1), (0, 4, 29), (80, 84, 92), (38, 79, 64), (88, 36, 63), (50, 65, 64), (72, 54, 93), (51, 65, 40), (11, 74, 7), (11, 25, 91), (4, 76, 51), (25, 29, 17), (36, 72, 66), (14, 14, 26), (13, 74, 4), (69, 13, 17), (88, 51, 93), (52, 46, 28), (91, 23, 92), (41, 72, 58), (41, 75, 3), (37, 94, 67), (85, 10, 61), (94, 47, 22), (74, 38, 24), (96, 52, 94), (46, 50, 96), (29, 38, 1), (24, 22, 87), (28, 93, 48), (89, 97, 2), (0, 5, 10), (20, 67, 71), (72, 0, 3)] 

Mapped Data:  [((12,), (12,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((1,), (1,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [27]:

def MAP(t):
  return (t, t)

def REDUCE(key, values):
  return (key, key)

def GROUP_BY_KEY(iterable):
  t = defaultdict(list)
  for k, v in iterable:
    t[k].append(v)
  return t.items()

def RECORDREADER(count):
  return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]


records = RECORDREADER(5)
print("Records: ", records, '\n')

map_res = list(map(lambda x: MAP(x), records))
print("Mapped Data: ", map_res, '\n')

group_res = list(GROUP_BY_KEY(map_res))
print("Shuffled Data: ", group_res, '\n')

reduce_res = list(map(lambda x: REDUCE(x[0], x[1])[0], group_res))
print("Reduced Data: ", reduce_res, '\n')

Records:  [(97, 83, 41), (60, 87, 56), (41, 8, 57), (59, 35, 90), (61, 91, 25)] 

Mapped Data:  [((97, 83, 41), (97, 83, 41)), ((60, 87, 56), (60, 87, 56)), ((41, 8, 57), (41, 8, 57)), ((59, 35, 90), (59, 35, 90)), ((61, 91, 25), (61, 91, 25))] 

Shuffled Data:  [((97, 83, 41), [(97, 83, 41)]), ((60, 87, 56), [(60, 87, 56)]), ((41, 8, 57), [(41, 8, 57)]), ((59, 35, 90), [(59, 35, 90)]), ((61, 91, 25), [(61, 91, 25)])] 

Reduced Data:  [(97, 83, 41), (60, 87, 56), (41, 8, 57), (59, 35, 90), (61, 91, 25)] 



### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [29]:
def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)]

def MAP(tuples):
    return [(t, t) for t in tuples]

def group_by_key(mapped_values):
    grouped_values = {}
    for key, value in mapped_values:
        if key in grouped_values:
            grouped_values[key].append(value)
        else:
            grouped_values[key] = [value]
    return grouped_values

def REDUCE(grouped_values):
    result = []
    for key, values in grouped_values.items():
        if len(values) > 1:  
            result.append((key, key))
    return result


input_data = RECORDREADER(3)


mapped_values = MAP(input_data)


grouped_values = group_by_key(mapped_values)


reduced_values = REDUCE(grouped_values)

print("Входные данные:", input_data)
print("Результат работы REDUCE:", reduced_values)

Входные данные: [(3, 1), (0, 1), (3, 1)]
Результат работы REDUCE: [((3, 1), (3, 1))]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [30]:
R = [5, 7]

def RECORDREADER(count):
    lst = []
    for i in range(count):
        data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
        idx = R[random.randint(0, len(R) - 1)]
        lst.append((data, idx))
    return lst

def MAP(t):
  return (t[0], t[1])

def REDUCE(key, values):
  if values == [R[0]]:
    return ((key[0], key[0]), (key[1], key[1]))

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()


records = RECORDREADER(50)
print("Records: ", records, '\n')

map_res = list(map(lambda x: MAP(x), records))
print("Mapped Data: ", map_res, '\n')

group_res = list(group_by_key(map_res))
print("Shuffled Data: ", group_res, '\n')

reduce_res = [element[0] for element in list(map(lambda x: REDUCE(*x), group_res)) if element is not None]
print("Reduced Data: ", reduce_res, '\n')

Records:  [(((3, 1), (2, 3), (3, 0), (2, 3), (1, 2), (1, 2), (3, 0), (2, 2), (1, 0), (1, 0), (2, 2), (3, 1), (2, 0), (0, 0), (1, 1), (2, 1), (2, 2), (1, 1), (3, 2), (2, 1), (3, 0), (2, 3), (1, 0), (3, 2), (1, 3), (3, 0), (1, 1), (1, 3), (3, 3), (3, 2), (2, 2), (0, 2), (1, 2), (1, 2), (1, 2), (2, 0), (3, 1), (3, 2), (3, 1), (2, 2), (3, 0), (3, 2), (1, 3), (0, 2), (2, 3), (3, 1), (0, 2), (2, 3), (0, 0), (2, 0)), 7), (((1, 3), (3, 3), (1, 2), (3, 2), (3, 0), (0, 2), (3, 2), (0, 1), (3, 2), (0, 2), (3, 0), (2, 0), (2, 0), (1, 1), (3, 2), (1, 2), (2, 2), (3, 0), (2, 2), (3, 2), (3, 2), (0, 1), (0, 1), (3, 2), (2, 1), (1, 0), (2, 3), (2, 0), (2, 1), (0, 1), (3, 3), (1, 3), (1, 2), (2, 2), (1, 1), (3, 2), (1, 3), (2, 1), (2, 1), (0, 1), (3, 2), (2, 2), (1, 3), (2, 1), (3, 1), (0, 3), (3, 0), (2, 3), (2, 1), (1, 2)), 5), (((1, 0), (2, 3), (2, 3), (3, 0), (1, 0), (0, 3), (2, 2), (1, 3), (2, 1), (0, 1), (0, 0), (3, 2), (3, 2), (2, 1), (0, 2), (0, 2), (2, 3), (1, 3), (3, 3), (0, 3), (1, 1), (3, 1

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [31]:
R = [5, 7]

def RECORDREADER(count):
    lst = []
    for i in range(count):
        data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
        idx = R[random.randint(0, len(R) - 1)]
        lst.append((data, idx))
    return lst

def MAP(t):
  data, idx = t
  if idx == R[0]:
    return (data[1], (idx, data[0]))
  else:
    return (data[0], (idx, data[1]))

def REDUCE(key, values):
  return [(v[0], key, v[1]) for v in values]

def group_by_key(iterable):
  return {k: list(v) for k, v in itertools.groupby(sorted(iterable), key=lambda x: x[0])}

records = RECORDREADER(50)
print("Records: ", records, '\n')

map_res = list(map(lambda x: MAP(x), records))
print("Mapped Data: ", map_res, '\n')

group_res = group_by_key(map_res)
print("Shuffled Data: ", group_res, '\n')

reduce_res = [REDUCE(k, v) for k, v in group_res.items()]
print("Reduced Data: ", reduce_res, '\n')

Records:  [(((1, 0), (0, 3), (1, 1), (0, 3), (0, 3), (2, 1), (0, 0), (1, 0), (1, 2), (3, 1), (2, 1), (2, 1), (1, 2), (3, 1), (2, 0), (0, 2), (1, 3), (0, 0), (2, 1), (3, 1), (3, 0), (0, 1), (2, 0), (2, 2), (3, 2), (1, 3), (2, 2), (0, 0), (3, 3), (3, 0), (3, 3), (2, 1), (1, 0), (0, 3), (3, 0), (0, 1), (3, 1), (3, 0), (3, 0), (2, 2), (3, 0), (3, 3), (2, 1), (0, 3), (3, 3), (0, 2), (3, 0), (3, 2), (0, 2), (0, 2)), 7), (((2, 2), (3, 2), (3, 3), (3, 0), (3, 3), (3, 3), (0, 2), (2, 1), (0, 0), (0, 2), (2, 2), (0, 2), (1, 3), (3, 2), (1, 0), (0, 1), (2, 1), (0, 1), (1, 2), (0, 0), (2, 2), (2, 3), (0, 1), (1, 1), (0, 3), (2, 2), (3, 2), (3, 0), (3, 3), (2, 1), (1, 0), (2, 3), (0, 1), (1, 1), (0, 3), (2, 2), (3, 2), (0, 0), (0, 0), (3, 3), (3, 0), (2, 3), (2, 1), (2, 0), (0, 1), (1, 1), (3, 1), (3, 1), (1, 3), (2, 2)), 7), (((3, 2), (2, 3), (3, 2), (3, 3), (2, 3), (2, 1), (3, 2), (1, 0), (1, 3), (1, 2), (1, 0), (2, 0), (1, 3), (0, 0), (2, 3), (3, 3), (3, 0), (2, 3), (1, 2), (3, 0), (2, 0), (3, 0

NameError: name 'itertools' is not defined

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [32]:
from typing import NamedTuple, Iterator, List, Tuple, Dict
from collections import defaultdict

class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    
input_collection = [
    User(id=0, age="55", gender='male', social_contacts=20),
    User(id=1, age="25", gender='female', social_contacts=240),
    User(id=2, age="25", gender='female', social_contacts=500),
    User(id=3, age="33", gender='female', social_contacts=800)
]

def map_function(user: User) -> Tuple[str, int]:
    return (user.age, user.social_contacts)
  
def reduce_function(age: str, social_contacts_list: List[int]) -> Tuple[str, int]:
    sum_social_contacts = sum(social_contacts_list)
    return (age, sum_social_contacts)

# Emulating MapReduce
def emulate_map_reduce(users: List[User]) -> List[Tuple[str, int]]:
    mapped = [map_function(user) for user in users]
    
    intermediate = defaultdict(list)
    for age, social_contacts in mapped:
        intermediate[age].append(social_contacts)
    
    reduced = [reduce_function(age, contacts) for age, contacts in intermediate.items()]
    
    return reduced


output = emulate_map_reduce(input_collection)
output

[('55', 20), ('25', 740), ('33', 800)]

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [33]:
import numpy as np
from collections import defaultdict

mat = np.ones((5, 4))
vec = np.random.rand(4)


vec_parts = {j: vec[j] for j in range(len(vec))}

def map_emulate(coordinates, value):
    i, j = coordinates
    # Доступ к соответствующей части вектора
    yield (i, value * vec_parts[j])

def reduce_emulate(i, products):
    sum_result = sum(products)
    yield (i, sum_result)

def record_reader_emulate():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield ((i, j), mat[i, j])


def emulate_map_reduce():
    map_results = defaultdict(list)
    

    for coordinates, value in record_reader_emulate():
        for result in map_emulate(coordinates, value):
            map_results[result[0]].append(result[1])
    

    reduce_results = []
    for i, products in map_results.items():
        for result in reduce_emulate(i, products):
            reduce_results.append(result)
    
    return reduce_results


output = emulate_map_reduce()
output

[(0, 1.691297266762131),
 (1, 1.691297266762131),
 (2, 1.691297266762131),
 (3, 1.691297266762131),
 (4, 1.691297266762131)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [34]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [35]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
  (j, k) = k1
  w = v1
  # solution code that yield(k2,v2) pairs
  for i in range(small_mat.shape[0]):
    yield ((i, k), small_mat[i, j] * w)
  
def REDUCE(key, values):
  (i, k) = key
  # solution code that yield(k3,v3) pairs
  yield (key, sum(values))

Проверьте своё решение

In [36]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [37]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [38]:
import numpy as np

# Инициализация размеров матриц
I, J, K = 2, 3, 40
# Инициализация матриц
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
    for i in range(I):
        for j in range(J):
            yield (('small', i, j), small_mat[i, j])
    for j in range(J):
        for k in range(K):
            yield (('big', j, k), big_mat[j, k])

def MAP(k1, v1):
    matrix_type, i_or_j, j_or_k = k1
    if matrix_type == 'small':
        for k in range(K):
            yield ((i_or_j, k), ('A', j_or_k, v1))
    else:
        for i in range(I):
            yield ((i, j_or_k), ('B', i_or_j, v1))

def REDUCE(key, values):
    sum_product = 0
    values_A = [v for v in values if v[0] == 'A']
    values_B = [v for v in values if v[0] == 'B']
    for a in values_A:
        for b in values_B:
            if a[1] == b[1]:
                sum_product += a[2] * b[2]
    yield (key, sum_product)

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        if k2 in t:
            t[k2].append(v2)
        else:
            t[k2] = [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    mapper_output = flatten([MAP(k1, v1) for k1, v1 in RECORDREADER()])
    grouped_items = groupbykey(mapper_output)
    reducer_output = flatten([REDUCE(key, values) for key, values in grouped_items])
    return reducer_output

reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)


def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true


True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [39]:
import itertools
import numpy as np
I = 2
J = 3
K = 40
first_mat = np.random.rand(I,J)
second_mat = np.random.rand(J,K)
reference_solution = np.matmul(first_mat, second_mat)

def INPUTFORMAT1():
    matrix1 = [((0, i, j), value) for (i, j), value in np.ndenumerate(first_mat)]
    yield matrix1

    matrix2 = [((1, j, k), value) for (j, k), value in np.ndenumerate(second_mat)]
    yield matrix2


def MAP1(k1, v1):
    mat_id, i, j = k1
    w = v1
    index = j if mat_id == 0 else i
    mat_idx = (mat_id, i, w) if mat_id == 0 else (mat_id, j, w)
    yield (index, mat_idx)


def REDUCE1(key, values):
    first_matr = [v for v in values if v[0] == 0]
    second_matr = [v for v in values if v[0] == 1]

    for f, s in itertools.product(first_matr, second_matr):
        yield ((f[1], s[1]), f[2] * s[2])


def INPUTFORMAT2():
  for j in for_inp:
    yield j[1]


def MAP2(k1, v1):
  yield (k1, v1)


def REDUCE2(key, value_list):
  total_value = sum(value_list)
  yield (key, total_value)


mr1 = MapReduceDistributed(INPUTFORMAT1, MAP1, REDUCE1, COMBINER=None)
for_inp = [(partition_id, list(partition)) for (partition_id, partition) in mr1]


mr2 = MapReduceDistributed(INPUTFORMAT2, MAP2, REDUCE2, COMBINER=None)
res = [(partition_id, list(partition)) for (partition_id, partition) in mr2]

result = [v for r in res for v in r[1]]


np.allclose(reference_solution, asmatrix(result)) # should return true

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [40]:
import itertools
import numpy as np
I = 2
J = 3
K = 40
first_mat = np.random.rand(I,J)
second_mat = np.random.rand(J,K)
reference_solution = np.matmul(first_mat, second_mat)
maps = 4
def INPUTFORMAT1():
  for mat_num, matrix in enumerate([first_mat, second_mat]):
    mat = [((mat_num, i, j), matrix[i, j]) for i, j in np.ndindex(matrix.shape)]
    split_size = int(np.ceil(len(mat) / maps))
    yield from (mat[i:i + split_size] for i in range(0, len(mat), split_size))


def MAP1(k1, v1):
    mat_id, i, j = k1
    w = v1
    index = j if mat_id == 0 else i
    mat_idx = (mat_id, i, w) if mat_id == 0 else (mat_id, j, w)
    yield (index, mat_idx)


def REDUCE1(key, values):
    first_matr = [v for v in values if v[0] == 0]
    second_matr = [v for v in values if v[0] == 1]

    for f, s in itertools.product(first_matr, second_matr):
        yield ((f[1], s[1]), f[2] * s[2])


def INPUTFORMAT2():
  for j in for_inp:
    yield j[1]


def MAP2(k1, v1):
  yield (k1, v1)


def REDUCE2(key, value_list):
  total_value = sum(value_list)
  yield (key, total_value)


mr1 = MapReduceDistributed(INPUTFORMAT1, MAP1, REDUCE1, COMBINER=None)
for_inp = [(partition_id, list(partition)) for (partition_id, partition) in mr1]


mr2 = MapReduceDistributed(INPUTFORMAT2, MAP2, REDUCE2, COMBINER=None)
res = [(partition_id, list(partition)) for (partition_id, partition) in mr2]

result = [v for r in res for v in r[1]]


np.allclose(reference_solution, asmatrix(result)) # should return true

True